Preprocess continuous data for resting state project. Updated pipeline integrating LFPAnalysis automatic IED rejection. Oct 2024.

Load necessary packages

In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import emd
import re
from ast import literal_eval

import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append('/Users/christinamaher/Documents/GitHub/LFPAnalysis')

In [3]:
from LFPAnalysis import lfp_preprocess_utils

Define directories

In [4]:
subj_id = 'MS009'
subj_format = ['edf']
subj_site = ['MSSM']

# Specify root directory for data and results 
base_dir = '/Users/christinamaher/Desktop/old_preprocess/'
anat_dir = f'{base_dir}{subj_id}/'
neural_dir = f'{base_dir}{subj_id}/'
save_dir = f'{base_dir}preprocess/clean_data/{subj_id}'
os.makedirs(save_dir,exist_ok = True) #added so you don't have to manually make subject folders in clean_data

Import data

In [5]:
edf_files = glob(f'{neural_dir}/*.edf')

mne_data = mne.io.read_raw_edf(edf_files[0], preload=True)
mne_data

<RawEDF | MS009.edf, 276 x 1838016 (1794.9 s), ~3.78 GB, data loaded>

Import anat recon information

In [6]:
# Load the electrode localization data
anat_file = glob(f'{anat_dir}/{subj_id}_labels.csv')[0]
elec_locs = pd.read_csv(anat_file)
# Sometimes there's extra columns with no entries: 
elec_locs = elec_locs[elec_locs.columns.drop(list(elec_locs.filter(regex='Unnamed')))]
elec_locs = elec_locs.dropna(axis=0, how = 'all') #some recons have a bunch of empty rows at the end 
elec_locs

,label,BN246label,x,y,z,mni_x,mni_y,mni_z,gm,NMM,Anat,AnatMacro,BN246,YBA_1,ManualExamination
0,LaCaS1,A32sg_L,-2.198512,42.942225,0.399996,-2.171989,31.060445,-9.573366,Gray,Left ACgG anterior cingulate gyrus,Area s24,L ACC,L CG,Left cingulate gyrus C,NaN
1,LaCaS10,A9l_L,-12.191809,54.538567,42.799993,-12.589239,50.871375,37.637815,White,Left SFG superior frontal gyrus,Unknown,L Superior Frontal Gyrus,L SFG,Left superior frontal gyrus 2 C,WM
2,LaCaS11,A9l_L,-13.391005,56.138062,47.999993,-13.741458,53.387483,43.456687,Unknown,Left SFG superior frontal gyrus,Unknown,L Superior Frontal Gyrus,L SFG,Unknown,OOB
3,LaCaS12,Unknown,-15.389664,57.337684,51.599993,-15.762398,55.196659,47.463093,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,OOB
4,LaCaS2,A32sg_L,-3.397708,44.141846,5.199996,-3.528236,33.282432,-4.187506,Gray,Left ACgG anterior cingulate gyrus,Area s24,L ACC,L CG,Left cingulate gyrus D,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,LpTpT7,A20cl_L,-59.759902,-25.836078,-8.400004,-58.231368,-43.255961,-9.333006,White,Left MTG middle temporal gyrus,Unknown,L Middle Temporal Gyrus,L ITG,Left inferior middle temporal gyrus E,WM
98,LpTpT8,A21c_L,-64.556684,-26.635826,-7.600004,-63.062459,-44.205698,-8.202658,White,Left MTG middle temporal gyrus,Unknown,L Middle Temporal Gyrus,L MTG,Left inferior middle temporal gyrus E,WM
99,LpTpT9,A37dl_L,-69.753199,-27.435573,-6.800004,-68.357228,-45.185994,-7.211913,Gray,Left MTG middle temporal gyrus,Unknown,L Middle Temporal Gyrus,L MTG,Left inferior middle temporal gyrus E,NaN
100,uLcPpP,A23d_L,-6.995295,-25.836078,21.599995,-5.040701,-35.483679,26.399148,Gray,Left PCgG posterior cingulate gyrus,Unknown,L PCC,L CG,Left cingulate gyrus O,NaN


Fix edf channel names

In [7]:
new_mne_names, unmatched_names, unmatched_seeg = lfp_preprocess_utils.match_elec_names(mne_data.ch_names, elec_locs.label)
new_name_dict = {x:y for (x,y) in zip(mne_data.ch_names, new_mne_names)}
mne_data.rename_channels(new_name_dict)


Number of electrodes in the mne file is greater than the number of electrodes in the localization file


<RawEDF | MS009.edf, 276 x 1838016 (1794.9 s), ~3.78 GB, data loaded>

In [8]:
anat_names = list(elec_locs.label.str.lower())
sum([ch not in mne_data.ch_names for ch in anat_names]) #if there are no missing channels, sum = 0. if sum >0, find the missing elecs
print([ch for ch in mne_data.ch_names if ch not in anat_names ]) #print extra channels in mne_data.ch_names and make sure none of them are neural channels (will be EEG etc.)

['c10', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64', 'c110', 'c111', 'c112', 'c113', 'c114', 'c115', 'c116', 'c117', 'c118', 'c119', 'c120', 'c121', 'c122', 'c123', 'c124', 'c125', 'c126', 'c127', 'c128', 'fp1', 'f7', 't3', 't5', 'o1', 'f3', 'c3', 'p3', 'fp2', 'f8', 't4', 't6', 'o2', 'f4', 'c4', 'p4', 'fz', 'cz', 'pz', 'ekg1', 'ekg2', 'c150', 'c151', 'c152', 'c153', 'c154', 'c155', 'c156', 'c157', 'c158', 'c159', 'c160', 'c161', 'c162', 'c163', 'c164', 'c165', 'c166', 'c167', 'c168', 'c169', 'c170', 'c171', 'c172', 'c173', 'c174', 'c175', 'c176', 'c177', 'c178', 'c179', 'c180', 'c181', 'c182', 'c183', 'c184', 'c185', 'c186', 'c187', 'c188', 'c189', 'c190', 'c191', 'c192', 'c193', 'c194', 'c195', 'c196', 'c197', 'c198', 'c199', 'c200', 'c201', 'c202', 'c203', 'c204', 'c205', 'c206', 'c207', 'c208', 'c209', 'c210', 'c211', 'c212', 'c213', 'c214', 'c215', 'c216', 'c217', 'c218', 'c219', 'c220', 'c221', 'c222', 'c223', 'c224', 'c225', 'c226', 'c227', 'c228', 'c229', 'c230', 'c23

In [9]:
# Note, there is surface EEG data that we should separately indicate from the sEEG:
right_seeg_names = [i for i in mne_data.ch_names if i.startswith('r')]
left_seeg_names = [i for i in mne_data.ch_names if i.startswith('l')]
print(f'We have a total of', len(left_seeg_names), 'left &', len(right_seeg_names), 'right sEEG electrodes')
print(f'We have a total of {len(left_seeg_names) + len(right_seeg_names)} sEEG electrodes')

We have a total of 100 left & 0 right sEEG electrodes
We have a total of 100 sEEG electrodes


In [10]:
drop_chans = list(set(mne_data.ch_names)^set(left_seeg_names+right_seeg_names)) # it is either called DC1 or research
mne_data.drop_channels(drop_chans) #number of chans should = number of seegs 

<RawEDF | MS009.edf, 100 x 1838016 (1794.9 s), ~1.37 GB, data loaded>

In [11]:
# Set channel types:
sEEG_mapping_dict = {f'{x}':'seeg' for x in left_seeg_names+right_seeg_names}
mne_data.set_channel_types(sEEG_mapping_dict)


<RawEDF | MS009.edf, 100 x 1838016 (1794.9 s), ~1.37 GB, data loaded>

In [12]:
# make montage (convert mm to m)

montage = mne.channels.make_dig_montage(ch_pos=dict(zip(elec_locs.label, 
                                                        elec_locs[['mni_x', 'mni_y', 'mni_z']].to_numpy(dtype=float)/1000)),
                                        coord_frame='mni_tal')

mne_data.set_montage(montage, match_case=False, on_missing='warn')

<RawEDF | MS009.edf, 100 x 1838016 (1794.9 s), ~1.37 GB, data loaded>

Notch filter line noise

In [13]:
# Identify line noise
mne_data.info['line_freq'] = 60

# Notch out 60 Hz noise and harmonics 
mne_data.notch_filter(freqs=(60, 120, 180, 240))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.9s finished


<RawEDF | MS009.edf, 100 x 1838016 (1794.9 s), ~1.37 GB, data loaded>

Resample data

In [14]:
#resampling if patient is not sampled at 512
resample_sr = 500
mne_data.resample(sfreq=resample_sr, npad='auto', n_jobs=-1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    8.7s finished


<RawEDF | MS009.edf, 100 x 897469 (1794.9 s), ~684.8 MB, data loaded>

Save raw lfp data

In [ ]:
mne_data.save(f'{save_dir}/{subj_id}_raw_ieeg.fif',overwrite=True)

Re-reference data - bipolar re-referencing 
- automatically excludes OOB channels

In [15]:
# Re-reference neural data
mne_data_bp_reref = lfp_preprocess_utils.ref_mne(mne_data=mne_data, 
                                              elec_path=f'{anat_dir}/{subj_id}_labels.csv', 
                                              method='bipolar', 
                                              site='MSSM')
mne_data_bp_reref

Number of electrodes in the mne file is less than the number of electrodes in the localization file


<RawEDF | MS009.edf, 60 x 897469 (1794.9 s), ~410.9 MB, data loaded>

In [ ]:
# Examine re-referenced data
%matplotlib qt

fig = mne_data_bp_reref.plot(start=0, duration=1000, n_channels=40, scalings=mne_data_bp_reref._data.max())

mne_data_bp_reref.compute_psd().plot()

Save bp-referenced data

In [ ]:
mne_data_bp_reref.save(f'{save_dir}/{subj_id}_raw_ieeg_reref.fif',overwrite=True)

Apply automatic artifact rejection 
- TODO check how this handle OOB channels etc. 

In [16]:
IED_args = {'peak_thresh':4,
           'closeness_thresh':0.25, 
           'width_thresh':0.2}

data_type = 'continuous'

IED_sec_dict = lfp_preprocess_utils.detect_IEDs(mne_data_bp_reref, 
                                            peak_thresh=IED_args['peak_thresh'], 
                                            closeness_thresh=IED_args['closeness_thresh'], 
                                            width_thresh=IED_args['width_thresh'])

artifact_sec_dict = lfp_preprocess_utils.detect_misc_artifacts(mne_data_bp_reref, 
                                            peak_thresh=IED_args['peak_thresh'])   

IED_df = pd.DataFrame.from_dict(IED_sec_dict, orient='index').T
artifact_df = pd.DataFrame.from_dict(artifact_sec_dict, orient='index').T 

### TODO: confirm with Salman it is ok to drop from raw signal instead of power. 
# Now, let's iterate through each channel, and each ied/artifact, and NaN 100 ms before and after these timepoints
for ch_ix, ch_name in enumerate(mne_data_bp_reref.ch_names): 
    ied_ev_list = IED_df[ch_name].dropna().index.tolist()
    artifact_ev_list = artifact_df[ch_name].dropna().index.tolist() 
    for ev_ in ied_ev_list: 
        for ied_ in literal_eval(IED_df[ch_name].iloc[ev_]):
            # remove 100 ms before 
            ev_ix_start = np.max([0, np.floor((ied_- 0.1) * mne_data_bp_reref.info['sfreq'])]).astype(int)
            # remove 100 ms after 
            ev_ix_end = np.min([mne_data_bp_reref._data.shape[-1], np.ceil((ied_ + 0.1) * mne_data_bp_reref.info['sfreq'])]).astype(int)
            mne_data_bp_reref._data.data[ch_ix, :, ev_ix_start:ev_ix_end] = np.nan
    for ev_ in artifact_ev_list: 
        for artifact_ in literal_eval(artifact_df[ch_name].iloc[ev_]):
            # remove 100 ms before 
            ev_ix_start = np.max([0, np.floor((artifact_- 0.1) * mne_data_bp_reref.info['sfreq'])]).astype(int)
            # remove 100 ms after
            ev_ix_end = np.min([mne_data_bp_reref._data.shape[-1], np.ceil((artifact_ + 0.1) * mne_data_bp_reref.info['sfreq'])]).astype(int)
            mne_data_bp_reref._data[ch_ix, :, ev_ix_start:ev_ix_end] = np.nan


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    1.9s finished


In [ ]:
# Save the cleaned data
mne_data_bp_reref.save(f'{save_dir}/{subj_id}_raw_ieeg_reref_cleaned.fif',overwrite=True)

Because this pipeline is fully automatic - we can parallelize it across subjects. 